In [45]:
import numpy as np
import pandas as pd
import pylab as pl
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [46]:
df = pd.read_csv('fcc_nyc.csv')

In [47]:
df1 = df.drop(['Business','MaxCIRDown','MaxCIRUp','HoldingCompanyName','HocoNum',
              'HocoFinal'],1).copy()

In [48]:
# remove satellite data as according to FCC, 
# satellite providers can simply check a single field and
# they are automatically attributed as providers for every census block in a given state, 
# which conflates analysis of actual access
df1 = df1[df1.TechCode != 60].copy()

# convert techcode to categorical label
platform = {0: 'Other', 10: 'Copper', 11:'Copper', 20: 'Copper', 30: 'Copper', 
            40: 'Cable', 41: 'Cable',42: 'Cable',50: 'Fiber', 
            60: 'Satellite', 70: 'Fixed_wireless'}

df1['TechCode'] = df1.TechCode.map(lambda x: platform[x])

In [49]:
# new df with aggregated values

df2 = pd.DataFrame(index=df1.FIPS.unique())
df2['avg_down'] = df1.groupby('FIPS')['MaxAdDown'].mean()
df2['avg_up'] = df1.groupby('FIPS')['MaxAdUp'].mean()
df2["max_down"] = df1.groupby('FIPS')['MaxAdDown'].max()
df2["max_up"] = df1.groupby('FIPS')['MaxAdUp'].max()
df2['num_providers'] = df1.groupby('FIPS')['Provider_Id'].nunique()
df2['num_platforms'] = df1.groupby('FIPS')['TechCode'].nunique()

In [50]:
# calculate top platform type and ratio for each census tract
top_plat = df1.groupby('FIPS')['TechCode'].value_counts(normalize=True).groupby(level=0).nlargest(1)

plat_type = []
top_ratio = []
for i in top_plat.index:
    plat_type.append(i[2])
    top_ratio.append(top_plat[i])
    
df2['top_plat_type'] = plat_type
df2['top_plat_ratio'] = top_ratio

In [51]:
# formula for broadband score: full weight for avg down, .5 weight for avg up, .1 weight for maxdown,
# which should be reflected in avg down, but this rewards tracts with high max.
# Same for number of provider: this is captured by average statistic, 
# but provides additional reward for having more providers regardless of speed

def bscore(data):
    return data.avg_down + .5*data.avg_up + .5*data.num_providers + .1*data.max_down

df2['bscore_raw'] = df2.apply(bscore, axis=1)
df2['bscore_norm'] = (df2.bscore_raw - df2.bscore_raw.mean())*1.0/df2.bscore_raw.std()

In [52]:
df2.reset_index(level=0, inplace=True)
df2=df2.rename(columns = {'index':'FIPS'})

In [55]:
df2.to_csv('broadband_features_by_tract.csv',index=False)